In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/08 21:15:22 WARN Utils: Your hostname, Mac-Pro-Olga.local resolves to a loopback address: 127.0.0.1; using 192.168.0.161 instead (on interface en0)
23/03/08 21:15:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 21:15:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/08 21:15:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/08 21:15:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv


--2023-03-08 21:15:25--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Распознаётся nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)… 3.5.25.164, 52.217.133.65, 52.217.79.228, ...
Подключение к nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|3.5.25.164|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 403 Forbidden
2023-03-08 21:15:26 ОШИБКА 403: Forbidden.



In [5]:
!wc -l fhvhv_tripdata_2021-01.csv


 11908469 fhvhv_tripdata_2021-01.csv


In [7]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [6]:


!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv


import pandas as pd
df_pandas = pd.read_csv('head.csv')
df_pandas.dtypes
spark.createDataFrame(df_pandas).schema


from pyspark.sql import types
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')
df = df.repartition(24)
df.write.parquet('fhvhv/2021/01/')
df = spark.read.parquet('fhvhv/2021/01/')
df.printSchema()



from pyspark.sql import functions as F
df.show()

/usr/local/opt/apache-spark/libexec/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/opt/apache-spark/libexec/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


AnalysisException: path file:/Users/omorozova/BigData/data-engineering-zoomcamp/week_5_batch_processing/fhvhv/2021/01 already exists.

In [ ]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'
crazy_stuff('B02884')

crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

In [ ]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')

In [ ]:
!head -n 10 head.csv
